In [2]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
from datetime import datetime
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

# Λήψη NLTK πόρων (εφόσον δεν είναι ήδη κατεβασμένα)
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Αρχικοποίηση εργαλείων NLP
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Συνάρτηση επεξεργασίας κειμένου (tokenization, stemming/lemmatization, stop-word removal and special characters removal)
def preprocess_text(text):
    # Χωρίζει το κείμενο σε λέξεις μέσω της word_tokenize
    tokens = word_tokenize(text) 
    # Εφαρμόζει lemmatization σε κάθε λέξη
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens] 
    # Φιλτράρει λέξεις, αφαιρώντας stopwords και special characters
    filtered_tokens = [token.lower().strip() for token in lemmatized_tokens if token.lower().strip() not in stop_words and token not in string.punctuation and token]
    # Ενώνει τις φιλτραρισμένες λέξεις σε ένα string
    processed_text = ''.join(filtered_tokens)
    return processed_text


def rank_documents(documents, query, similarity_threshold=0):
    # Εισαγωγή απαραίτητης βιβλιοθήκης για TF-IDF
    from sklearn.feature_extraction.text import TfidfVectorizer
    # Δημιουργία ενός vectorizer TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    # Υπολογισμός των TF-IDF τιμών για τα έγγραφα
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    # Μετατροπή του ερωτήματος σε TF-IDF διάνυσμα
    query_vector = tfidf_vectorizer.transform([query])
    # Κατάταξη των εγγράφων με βάση τις τιμές TF-IDF σε σχέση με το ερώτημα
    scores = tfidf_matrix.dot(query_vector.T).toarray().flatten()
    # Φιλτράρισμα των εγγράφων με βάση το κατώφλι ομοιότητας
    results = [(documents[i], scores[i]) for i in range(len(documents)) if scores[i] > similarity_threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    return results

# Αλγόριθμος Boolean retrieval
def boolean_retrieval(query, inverted_index, documents):
    # Διαχωρισμός των λέξεων του ερωτήματος
    query_terms = query.split()
    # Εύρεση σχετικών εγγράφων που περιέχουν όλους τους όρους του ερωτήματος
    relevant_docs = set.intersection(*(set(inverted_index.get(term, set())) for term in query_terms))
    return list(relevant_docs)

# Εφαρμογή λογικού τελεστή ανάμεσα σε δύο σύνολα
def apply_logical_operator(operand1, operand2, operator):
    if operator == 'AND':
        return set.intersection(operand1, operand2)
    elif operator == 'OR':
        return set.union(operand1, operand2)
    elif operator == 'NOT':
        return set.difference(operand1, operand2)

# Επεξεργασία ενός σύνθετου ερωτήματος με λογικούς τελεστές
def process_query(query, inverted_index, documents):
    # Διαχωρισμός των όρων του ερωτήματος
    query_terms = query.split()
    # Λίστα για την αποθήκευση των ενδιάμεσων αποτελεσμάτων
    processed_query = []
    logical_operator = None # Μεταβλητή για τον τρέχοντα λογικό τελεστή
    
    for term in query_terms:
        # Αν ο όρος είναι λογικός τελεστής, τον αποθηκεύουμε
        if term.upper() in ('AND', 'OR', 'NOT'):
            logical_operator = term.upper()
        else:
            # Εφαρμογή του λογικού τελεστή στα ενδιάμεσα αποτελέσματα
            if logical_operator and processed_query:
                processed_query[-1] = apply_logical_operator(processed_query[-1], set(boolean_retrieval(term, inverted_index, documents)), logical_operator)
                logical_operator = None
            # Αναζήτηση εγγράφων για τον τρέχοντα όρο      
            else:
                processed_query.append(set(boolean_retrieval(term, inverted_index, documents)))
                
    # Επεξεργασία του τελεστή NOT στη λίστα
    while 'NOT' in processed_query:
        not_index = processed_query.index('NOT')
        processed_query[not_index:not_index + 2] = apply_logical_operator(set(range(len(documents))), processed_query[not_index + 1], 'NOT')
    return processed_query

search = input("What Are You Looking For?")
search = search.replace(' ', '-')
url = f"https://arxiv.org/search/?searchtype=all&query={search}&abstracts=show&size=100&order=-announced_date_first"
page = requests.get(url).text
documents = BeautifulSoup(page, "html.parser")

# Βρίσκουμε το τμήμα HTML που περιέχει τα έγγραφα, αναζητώντας τη λίστα με την κλάση "breathe-horizontal"
div = documents.find("ol", {"class": "breathe-horizontal"})

# Δημιουργούμε μια λίστα για να αποθηκεύσουμε τις πληροφορίες κάθε εγγράφου
documents_list = []
# Διατρέχουμε κάθε στοιχείο "li" στη λίστα εγγράφων
for doc in div.find_all("li", recursive=False):
    # Εξαγωγή και καθαρισμός του τίτλου
    title = doc.find(class_="title is-5 mathjax").get_text(strip=True, separator=' ')
    # Εξαγωγή και καθαρισμός των συγγραφέων
    authors = doc.find(class_="authors").get_text(strip=True, separator=' ')
    # Εξαγωγή και καθαρισμός της περίληψης
    abstract = doc.find("span", {"class": "abstract-full has-text-grey-dark mathjax"}).get_text(strip=True, separator=' ')
    # Εξαγωγή και καθαρισμός πρόσθετων πληροφοριών, όπως η ημερομηνία
    additional_info = doc.find("p", {"class": "is-size-7"}).get_text(strip=True, separator=' ')
    date_text = additional_info.split(';')[0].strip().replace("Submitted","")
    original_date_object = datetime.strptime(date_text, " %d %B, %Y")
    formatted_date = original_date_object.strftime("%d/%m/%Y")
    
    # Δημιουργία ενός λεξικού με τις πληροφορίες του εγγράφου
    document_info = {
        "Title": title,
        "Authors": authors,
        "Abstract": abstract,
        "Sumbitted": formatted_date
    }

    # Προσθέτουμε το λεξικό στη λίστα εγγράφων
    documents_list.append(document_info)

# Αποθήκευση των πληροφοριών εγγράφων σε αρχείο κειμένου
with open("arxivdocstest.txt", "w", encoding="utf-8") as file:
    for idx, document_info in enumerate(documents_list, start=1):
        file.write(f"Document {idx}:\n")
        for key, value in document_info.items():
            file.write(f"{key}: {value}\n")
        file.write('='*50 + '\n')
 

# Κατεβάζουμε δεδομένα NLTK αν δεν είναι ήδη εγκατεστημένα
nltk.download('punkt')

# Κατακερματίζουμε το κείμενο (tokenization) για κάθε έγγραφο στη λίστα
tokenized_documents_list = []

for document_info in documents_list:
    tokenized_document_info = {}

    for key, value in document_info.items():
        # Χρήση της word_tokenize για κατακερματισμό του κειμένου
        tokens = word_tokenize(value)
        
        # Αποθήκευση των tokens στο λεξικό
        tokenized_document_info[key] = tokens

    # Προσθέτουμε το tokenized λεξικό στη λίστα
    tokenized_documents_list.append(tokenized_document_info)

# Αποθήκευση των tokenized πληροφοριών σε αρχείο κειμένου
with open("tokenized_arxivdocs.txt", "w", encoding="utf-8") as file:
    for idx, tokenized_document_info in enumerate(tokenized_documents_list, start=1):
        file.write(f"Tokenized Document {idx}:\n")
        for key, tokens in tokenized_document_info.items():
            file.write(f"{key}: {tokens}\n")
        file.write('='*50 + '\n')

        
# Επεξεργασία κάθε εγγράφου στη λίστα tokenized εγγράφων
preprocessed_documents_list = []

for tokenized_document_info in tokenized_documents_list:
    preprocessed_document_info = {}
    
    for key, tokens in tokenized_document_info.items():
        # Επεξεργασία των tokens και φιλτράρισμα κενών strings
        processed_tokens = [preprocess_text(token) for token in tokens if preprocess_text(token)]
        # Αποθήκευση των επεξεργασμένων tokens στο λεξικό
        preprocessed_document_info[key] = processed_tokens

    # Προσθέτουμε το επεξεργασμένο λεξικό στη λίστα
    preprocessed_documents_list.append(preprocessed_document_info)

# Αποθήκευση των επεξεργασμένων πληροφοριών σε αρχείο κειμένου
with open("preprocessed_arxivdocs.txt", "w", encoding="utf-8") as file:
    for idx, preprocessed_document_info in enumerate(preprocessed_documents_list, start=1):
        file.write(f"Preprocessed Document {idx}:\n")
        for key, processed_tokens in preprocessed_document_info.items():
            file.write(f"{key}: {processed_tokens}\n")
        file.write('='*50 + '\n')
        
        
# Ανεστραμμένη δομή δεδομένων ευρετηρίου (inverted index)
inverted_index = {}

for idx, preprocessed_document_info in enumerate(preprocessed_documents_list, start=1):
    for key, processed_tokens in preprocessed_document_info.items():
        for token in processed_tokens:
            if token not in inverted_index:
                inverted_index[token] = []
            inverted_index[token].append(idx)


# Εξαγωγή και αποθήκευση του ανεστραμμένου ευρετηρίου (inverted index) σε αρχείο κειμένου
with open("inverted_index.txt", "w", encoding="utf-8") as file:
    for term, document_ids in inverted_index.items():
        file.write(f"{term}: {document_ids}\n")

# Άνοιγμα του αρχείου "arxivdocstest.txt" που περιέχει τις αποθηκευμένες πληροφορίες εγγράφων
with open('arxivdocstest.txt', encoding="utf8") as arxivdocstest:
    sent1 = arxivdocstest.read()
    sent1sp = re.split(r'={50,}', sent1)
    
    
# Ζητάμε από τον χρήστη να εισάγει το ερώτημα και τι αλγόριθμος θέλει να χρησιμοποιηθεί
query = input("Enter The Query: ")
alg = input("\nWhat Type Of Retrieval You Want To Use? \n1)Boolean Retrieval \n2)Vector Space Model \n3)Okapi BM25\n")
# Boolean Retrieval
if alg == "1":
        # Επεξεργασία του ερωτήματος με χρήση του Boolean Retrieval αλγορίθμου
        result = process_query(query, inverted_index, documents)
        print("\nRetrieved Results:")
        res_list = []
        for doc_set in result:
            for doc_id in doc_set:
                print(sent1sp[doc_id-1])
                res=sent1sp[doc_id-1]
                res_list.append(res)
# Vector Space Model (VSM)          
elif alg == "2":
    # Υπολογισμός της ομοιότητας και κατάταξη των εγγράφων
    results = rank_documents(sent1sp, query)
    # Εκτύπωση των ταξινομημένων εγγράφων με βάση την ομοιότητα
    for doc, similarity in results:
        print(f"Similarity: {similarity:.2f}\n{doc}\n")
# Okapi BM25             
elif alg == "3":
    # Προετοιμασία δεδομένων για BM25
    # Tokenize τα έγγραφα
    tokenized_corpus = [word_tokenize(doc.lower()) for doc in sent1sp]
    # Δημιουργία αντικειμένου BM25
    bm25 = BM25Okapi(tokenized_corpus)
    # Tokenize το query
    tokenized_query = word_tokenize(query.lower())
    # Υπολογισμός βαθμολογιών συνάφειας
    scores = bm25.get_scores(tokenized_query)
    # Ταξινόμηση εγγράφων με βάση τις βαθμολογίες
    ranked_results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    # Εμφάνιση αποτελεσμάτων
    print("\nRetrieved Results (Ranked):")
    for idx, score in ranked_results:
        if score > 0:  # Εμφάνιση μόνο σχετικών εγγράφων
            print(f"Score: {score:.2f}\nDocument: {sent1sp[idx]}\n")
        else:
            break

else:
    print("Invalid algorithm selection.")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\difi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\difi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\difi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


What Are You Looking For? cat and dog


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\difi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Enter The Query:  cat and dog

What Type Of Retrieval You Want To Use? 
1)Boolean Retrieval 
2)Vector Space Model 
3)Okapi BM25
 1



Retrieved Results:
Document 1:
Title: Image Classification Using Singular Value Decomposition and Optimization
Authors: Authors: Isabela M. Yepes , Manasvi Goyal
Abstract: This study investigates the applicability of Singular Value Decomposition for the image classification of specific breeds of cats and dogs using fur color as the primary identifying feature. Sequential Quadratic Programming (SQP) is employed to construct optimally weighted templates. The proposed method achieves 69% accuracy using the Frobenius norm at rank 10. The results partially validate the assumption that dominant features, such as fur color, can be effectively captured through low-rank approximations. However, the accuracy suggests that additional features or methods may be required for more robust classification, highlighting the trade-off between simplicity and performance in resource-constrained environments. △ Less
Sumbitted: 10/12/2024


Document 3:
Title: Agent-to-Sim: Learning Interactive Behavior Mode